In [1]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/Shareddrives/RFID_23-24/Final Setup

/content/drive/Shareddrives/RFID_23-24/Final Setup


In [4]:
import os
import pandas as pd
import re

In [6]:


def process_csv_file(filename, directory):
    # Extract X and Y values from the filename
    match = re.search(r'X(\d+)Y(\d+)A(\d+)', filename)
    if match:
        x, y, a = map(int, match.groups())

        # Read the CSV file, skipping the first 2 rows
        fname = os.path.join(directory, filename)
        df = pd.read_csv(fname, skiprows=2)

        # Data processing steps
        drops = [" TID", " Frequency", " Hostname", " PhaseAngle", " DopplerFrequency"]
        df = df.drop(drops, axis=1)
        df = df.rename(columns={"// Timestamp": "Timestamp", " EPC": "EPC", " Antenna": "Antenna", " RSSI": "RSSI"})
        df['Time'] = df['Timestamp'].apply(lambda x: re.findall(r'\d+\.\d+', x)[0])

        # Calculate the number of unique tags and total read count
        num_unique_tags = df['EPC'].nunique()
        total_read_count = len(df)

        # Filter for target EPC
        target_df = df[df['EPC'] == 'E280689000000000E0BAFC73']
        read_count_target = len(target_df)


        new_df = df[df['EPC'] == 'E280689000000000E0BAFC73']
        new_df = new_df.drop(["Timestamp", 'EPC'], axis=1)
        new_df['Time'] = pd.to_numeric(new_df['Time'])
        new_df['Time'] = new_df['Time'].round()
        unique_values = new_df['Time'].unique()
        mask = (new_df['Time'] != unique_values[0]) & (new_df['Time'] != unique_values[-1])
        df_filtered = new_df[mask]
        mean_df = df_filtered.groupby(['Time', 'Antenna'])['RSSI'].mean().unstack()

        # Check the number of columns and rename accordingly
        antenna_columns = ['Antenna1', 'Antenna2', 'Antenna3', 'Antenna4']
        existing_columns = mean_df.columns
        rename_dict = {ant: ant_col for ant, ant_col in zip(existing_columns, antenna_columns)}
        mean_df = mean_df.rename(columns=rename_dict)

        # Fill missing columns with 0
        for col in antenna_columns:
            if col not in mean_df.columns:
                mean_df[col] = float(0)

        # Calculate mean and standard deviation
        mean_values = mean_df.mean(axis=0)
        std_values = mean_df.std(axis=0)

        # Create a DataFrame with the mean and standard deviation values
        result_df = pd.DataFrame({
            'A1_sd': [std_values['Antenna1']],
            'A1_mean': [mean_values['Antenna1']],
            'A2_sd': [std_values['Antenna2']],
            'A2_mean': [mean_values['Antenna2']],
            'A3_sd': [std_values['Antenna3']],
            'A3_mean': [mean_values['Antenna3']],
            'A4_sd': [std_values['Antenna4']],
            'A4_mean': [mean_values['Antenna4']],
            'angle': a,
            'x': x,
            'y': y
        })

        return result_df
    else:
        # Return None or an empty DataFrame if the filename pattern does not match
        return None



In [8]:
# Directory containing your CSV files
directory = 'P4'

# DataFrame to store the condensed data
condensed_df_p1 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P1_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P1_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p1 = pd.concat([condensed_df_p1, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p1)


        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    1.056977 -64.302011  1.598110 -58.971935  1.227933 -57.976416  1.893456   
1    1.294601 -62.292824  0.678158 -54.266620  0.661239 -56.842639  1.247256   
2    1.007785 -65.491092  0.310686 -52.964669  0.904539 -55.038472  0.953702   
3    0.826074 -64.844168  0.644847 -55.814484  0.387004 -56.614038  0.816568   
4    0.628132 -66.309568  0.608227 -50.370936  1.613931 -64.172811  1.344798   
..        ...        ...       ...        ...       ...        ...       ...   
116  0.875053 -64.568562  1.562406 -61.102244  0.966814 -60.059715  1.099878   
117  0.921540 -62.662452  0.947312 -50.468678  0.909721 -57.013745  1.074675   
118  0.317204 -67.392857  0.578601 -58.457102  0.521060 -59.233333  0.531223   
119  1.456722 -62.959921  1.696974 -58.268918  0.698608 -56.694444  1.167166   
120  0.590565 -62.574074  0.766605 -56.638241  0.599230 -55.957963  0.817552   

       A4_mean  angle   x   y  
0   -61

In [ ]:
# Directory containing your CSV files
directory = 'P2'

# DataFrame to store the condensed data
condensed_df_p2 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P2_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P2_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p2 = pd.concat([condensed_df_p2, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p2)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    0.455694 -66.878788  0.602957 -59.091898  1.126873 -59.759676  0.388383   
1    1.092510 -61.473898  1.880662 -59.411015  0.365428 -55.353161  0.000000   
2    2.064072 -61.387911  0.448910 -53.873030  0.396486 -52.027826  0.450729   
3    1.417068 -63.967837  1.179202 -58.408155  1.153554 -57.107136  1.088665   
4    0.571581 -66.878788  1.183826 -62.419752  1.526860 -61.282731  0.869344   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.388903 -62.096853  0.724441 -59.646073  0.416072 -59.488362  1.072401   
117  1.211035 -62.046928  0.648752 -55.072653  0.865243 -57.868487  1.714091   
118  1.282819 -64.610632  0.812236 -60.130302  1.033568 -58.058764  0.289868   
119  1.201266 -63.196783  1.276057 -57.733829  0.875569 -58.464286  1.033874   
120  1.079395 -64.774128  0.856707 -56.613790  0.959354 -58.171280  0.507117   

       A4_mean  x  y  
0   -55.083102  

In [ ]:
# Directory containing your CSV files
directory = 'P3'

# DataFrame to store the condensed data
condensed_df_p3 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P3_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P3_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p3 = pd.concat([condensed_df_p3, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p3)

       A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0   0.262295 -54.666296  0.893082 -57.163565  0.601628 -54.444769  0.390757   
1   0.437782 -56.294722  1.712325 -60.282824  0.344609 -50.840741  0.570835   
2   2.099158 -62.603915  0.251686 -54.011620  0.373481 -51.972454  0.479764   
3   0.565324 -56.941806  0.517637 -52.129583  0.504318 -57.070370  0.864280   
4   0.258062 -53.812356  0.644418 -55.480029  0.562981 -54.930172  0.586414   
5   0.464076 -55.921667  1.740017 -59.975086  0.352943 -51.463459  0.655055   
6   1.605513 -62.079276  0.346797 -55.178352  0.458394 -52.703209  0.560038   
7   0.286286 -57.743254  0.768364 -53.440823  0.550352 -58.481200  0.810704   
8   1.024918 -56.756842  0.892798 -62.325465  0.492037 -57.239163  0.382001   
9   1.358887 -63.156973  0.733043 -61.400730  1.076515 -57.993382  1.054651   
10  0.861421 -63.355988  0.588395 -56.049405  0.625856 -58.641667  0.858223   
11  0.816964 -55.525431  0.401679 -55.844397  0.8658

In [ ]:
# Directory containing your CSV files
directory = 'P4'

# DataFrame to store the condensed data
condensed_df_p4 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P4_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P4_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p4 = pd.concat([condensed_df_p4, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p4)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r4.csv'

# Export the DataFrame to a CSV file
condensed_df_p4.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    1.056977 -64.302011  1.598110 -58.971935  1.227933 -57.976416  1.893456   
1    1.294601 -62.292824  0.678158 -54.266620  0.661239 -56.842639  1.247256   
2    1.007785 -65.491092  0.310686 -52.964669  0.904539 -55.038472  0.953702   
3    0.826074 -64.844168  0.644847 -55.814484  0.387004 -56.614038  0.816568   
4    0.628132 -66.309568  0.608227 -50.370936  1.613931 -64.172811  1.344798   
..        ...        ...       ...        ...       ...        ...       ...   
116  0.875053 -64.568562  1.562406 -61.102244  0.966814 -60.059715  1.099878   
117  0.921540 -62.662452  0.947312 -50.468678  0.909721 -57.013745  1.074675   
118  0.317204 -67.392857  0.578601 -58.457102  0.521060 -59.233333  0.531223   
119  1.456722 -62.959921  1.696974 -58.268918  0.698608 -56.694444  1.167166   
120  0.590565 -62.574074  0.766605 -56.638241  0.599230 -55.957963  0.817552   

       A4_mean   x   y  
0   -61.379146

In [ ]:
# Directory containing your CSV files
directory = 'P5 Test'

# DataFrame to store the condensed data
condensed_df_p5 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P5_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P5_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p5 = pd.concat([condensed_df_p5, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p5)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r5.csv'

# Export the DataFrame to a CSV file
condensed_df_p5.to_csv(output_file_path, index=False)

       A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0   1.064916 -62.159143  0.345426 -50.237500  0.557548 -56.173611  0.934130   
1   0.581023 -67.134058  0.565905 -57.981513  1.169005 -55.828784  0.299721   
2   1.178610 -65.474830  0.388335 -54.330417  0.482054 -57.917361  1.191207   
3   1.207373 -63.543135  1.000057 -54.836045  0.889776 -53.994861  0.701560   
4   0.976794 -65.708169  0.678396 -61.037124  0.760380 -60.326539  0.459224   
5   1.057681 -62.306247  0.935678 -58.023727  0.828215 -60.227395  0.940583   
6   1.410227 -65.227941  1.024692 -62.790844  1.255034 -61.991619  0.297235   
7   0.456852 -62.614908  0.729468 -56.574952  0.516027 -56.307663  0.728091   
8   0.730513 -62.805083  0.493981 -53.597605  0.541298 -54.399665  0.563327   
9   2.759964 -63.630511  0.731179 -51.269636  0.555526 -57.480795  0.682019   
10  1.188078 -63.298241  0.370805 -51.157963  0.588181 -52.474630  0.988033   
11  1.046596 -65.643849  1.414781 -61.979782  0.8040

In [ ]:
# Directory containing your CSV files
directory = 'P6'

# DataFrame to store the condensed data
condensed_df_p6 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P6_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P6_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p6 = pd.concat([condensed_df_p6, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p6)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r6.csv'

# Export the DataFrame to a CSV file
condensed_df_p6.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    0.753683 -61.175000  0.566482 -49.546073  1.010083 -57.424090  1.004096   
1    1.085227 -66.622024  0.982774 -59.911643  1.236516 -62.471188  1.266908   
2    0.846406 -65.141441  1.676023 -59.097725  0.730210 -61.551554  1.544445   
3    2.235616 -65.652976  0.915466 -52.036951  1.145033 -56.497593  0.929424   
4    1.250084 -61.656322  1.755289 -59.963451  0.826185 -60.497701  1.037061   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.324725 -63.400151  1.276800 -57.313684  0.437633 -53.010393  0.956390   
117  1.685640 -64.779167  1.055953 -59.377169  1.837299 -56.523077  0.882110   
118  1.032504 -62.228551  0.550866 -52.993151  0.725054 -53.462069  0.855047   
119  0.997806 -65.268493  1.037976 -60.381760  1.175193 -55.497174  0.686819   
120  0.874922 -66.118534  1.945830 -59.867365  1.417389 -58.525503  1.426153   

       A4_mean  x   y  
0   -61.507232 

In [ ]:
# Directory containing your CSV files
directory = 'P7'

# DataFrame to store the condensed data
condensed_df_p7 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P7_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P7_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p7 = pd.concat([condensed_df_p7, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p7)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r7.csv'

# Export the DataFrame to a CSV file
condensed_df_p7.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    1.188981 -62.747092  0.497434 -61.117700  0.624975 -57.976136  0.563261   
1    2.068468 -63.491429  1.106155 -60.037824  1.048703 -59.338915  0.000000   
2    1.033698 -64.081439  1.492177 -62.332474  0.777344 -59.848538  0.263627   
3    0.852025 -63.449514  0.857079 -58.766284  0.431955 -57.695115  0.964125   
4    1.136681 -62.913095  0.638707 -53.077675  1.601531 -54.565086  0.977412   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.015270 -63.581888  1.030396 -55.391831  0.439832 -53.759004  1.326878   
117  1.447379 -62.263948  1.286012 -63.897024  2.172254 -60.144173  0.997050   
118  0.972983 -61.164819  0.850624 -55.544554  0.352577 -50.171750  0.308949   
119  0.697999 -64.706000  0.881815 -60.956260  1.070688 -55.196654  0.259996   
120  1.323097 -61.464566  0.495399 -52.852969  1.343660 -52.635105  1.054805   

       A4_mean  x  y  
0   -55.244157  

In [ ]:
# Directory containing your CSV files
directory = 'P8'

# DataFrame to store the condensed data
condensed_df_p8 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P8_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P8_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p8 = pd.concat([condensed_df_p8, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p8)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r8.csv'

# Export the DataFrame to a CSV file
condensed_df_p8.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    0.955593 -61.162851  1.314664 -64.256052  1.305377 -61.674385  1.301782   
1    1.418293 -63.268726  1.201564 -62.031185  0.937489 -64.749507  0.910085   
2    2.017317 -62.142432  1.803726 -62.038116  0.765898 -58.982115  1.193487   
3    1.105299 -58.750794  1.270511 -62.934488  0.872228 -59.654365  1.120035   
4    2.534926 -62.628288  1.451718 -62.883284  1.512786 -58.429117  1.102047   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.356871 -56.194096  1.651618 -63.744964  0.661665 -55.000623  0.778585   
117  0.829386 -60.919684  1.882484 -62.519071  1.116388 -64.243569  1.061029   
118  1.185217 -62.349412  0.437715 -65.995284  1.393271 -59.848058  1.639170   
119  1.061492 -58.267143  1.128438 -65.136657  0.863940 -62.911620  1.004302   
120  1.085732 -54.853333  1.409750 -62.187131  1.154649 -60.509676  0.703777   

       A4_mean   x   y  
0   -64.599861

In [ ]:
# Directory containing your CSV files
directory = 'P9'

# DataFrame to store the condensed data
condensed_df_p9 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P9_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P9_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p9 = pd.concat([condensed_df_p9, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p9)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r9.csv'

# Export the DataFrame to a CSV file
condensed_df_p9.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    1.343372 -62.837719  0.898426 -63.154953  1.230474 -62.904475  1.365690   
1    1.665139 -58.907044  0.477067 -65.826637  0.942993 -58.455804  0.725796   
2    1.018132 -61.531110  1.419760 -64.615661  1.026618 -62.076334  1.112286   
3    1.941631 -60.114368  1.266967 -63.371189  1.107817 -62.251676  1.148329   
4    0.908954 -61.868343  1.169936 -61.727278  0.888141 -63.556890  0.706920   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.227858 -62.479425  1.420081 -61.928611  1.199596 -62.727983  0.850616   
117  1.217703 -57.937924  1.006017 -60.516071  0.646725 -57.473228  0.519049   
118  1.140658 -59.143918  1.381853 -62.491305  1.031579 -62.302155  0.689674   
119  0.559049 -57.627083  1.141852 -62.729782  1.428310 -63.250060  0.293040   
120  1.614217 -59.116172  1.000319 -61.697824  0.402574 -55.333241  0.447206   

       A4_mean   x  y  
0   -61.866933 

In [ ]:
# Directory containing your CSV files
directory = 'P10'

# DataFrame to store the condensed data
condensed_df_p10 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P10_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P10_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p10 = pd.concat([condensed_df_p10, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p10)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r10.csv'

# Export the DataFrame to a CSV file
condensed_df_p10.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    1.817293 -60.468796  1.868850 -62.841913  0.910228 -62.931343  0.631550   
1    0.969640 -60.401528  1.401658 -61.559385  0.998420 -62.603287  1.039569   
2    1.974779 -61.115579  1.164875 -64.033929  1.264027 -62.601519  1.594338   
3    1.256177 -55.542411  1.084203 -60.164534  0.791760 -57.730357  0.919934   
4    0.797503 -53.204960  1.805566 -61.095770  0.708746 -58.481002  1.644707   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.156410 -61.732136  0.903945 -62.099239  0.831856 -62.588089  1.841389   
117  0.583086 -51.978333  1.588887 -58.940853  0.502906 -55.431343  0.668040   
118  0.831527 -55.620307  0.580550 -57.593391  0.779857 -55.485010  0.404336   
119  1.311268 -58.162535  1.013275 -63.179982  0.875831 -59.012550  1.532464   
120  1.431908 -56.996169  1.230145 -56.953736  1.613152 -60.745047  1.234892   

       A4_mean   x   y  
0   -64.664327

In [ ]:
# Directory containing your CSV files
directory = 'P11'

# DataFrame to store the condensed data
condensed_df_p11 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P11_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P11_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p11 = pd.concat([condensed_df_p11, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p11)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r11.csv'

# Export the DataFrame to a CSV file
condensed_df_p11.to_csv(output_file_path, index=False)

        A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0    0.583818 -50.867481  0.847748 -57.047126  1.145233 -59.524617  0.647966   
1    0.176505 -53.473065  0.828778 -59.507072  1.070547 -58.479216  1.554097   
2    0.349328 -52.557841  0.640935 -55.722031  1.030177 -55.808477  3.176320   
3    0.634920 -57.208194  1.213834 -60.288287  0.480912 -56.054583  0.773093   
4    1.281769 -54.080463  1.505660 -60.331984  0.931395 -58.527546  1.041656   
..        ...        ...       ...        ...       ...        ...       ...   
116  1.202640 -61.775509  0.879585 -59.486481  0.796642 -60.117639  0.884519   
117  1.960000 -57.986019  0.997752 -62.367231  1.878586 -60.338194  0.347137   
118  0.944575 -56.063602  1.545101 -60.063027  0.493124 -57.672605  0.669381   
119  0.970518 -55.724282  1.786930 -63.352682  0.863907 -60.154741  1.892440   
120  0.547047 -52.241481  1.020906 -59.032639  0.310053 -55.663843  0.630880   

       A4_mean  x  y  
0   -60.930056  

In [ ]:
# Directory containing your CSV files
directory = 'P12'

# DataFrame to store the condensed data
condensed_df_p12 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P12_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P12_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p12 = pd.concat([condensed_df_p12, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p12)

# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_r12.csv'

# Export the DataFrame to a CSV file
condensed_df_p12.to_csv(output_file_path, index=False)

       A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0   0.377964 -66.285714  0.212031 -53.544494  1.653587 -62.105719  0.669327   
1   1.150062 -62.428848  1.007721 -60.612124  0.878862 -61.902112  0.000000   
2   0.472787 -65.829365  1.124974 -57.281657  0.696453 -55.707423  1.490865   
3   1.401139 -57.558525  1.473461 -58.091188  1.597732 -60.115374  1.041990   
4   1.302094 -63.425079  0.646414 -53.225833  1.147283 -53.440370  1.379043   
5   0.720046 -60.241111  1.298204 -63.708882  1.155505 -62.418344  1.040509   
6   0.958443 -58.231630  0.841205 -64.531808  0.485091 -62.415787  1.449652   
7   1.061988 -59.831568  1.374140 -63.396067  1.341451 -64.338793  1.328535   
8   2.552826 -58.850000  0.967217 -62.152083  1.490223 -59.930371  0.323445   
9        NaN -67.000000  1.451265 -63.056034  1.400422 -60.804817  1.055943   
10  0.711873 -55.130699  0.505088 -57.877011  0.968533 -55.194588  0.357356   
11  0.582111 -66.526071  1.817151 -61.512646  1.0722

In [ ]:
condensed_data_3_28_4angles = pd.concat([condensed_df_p2, condensed_df_p4, condensed_df_p6, condensed_df_p7, condensed_df_p8, condensed_df_p9, condensed_df_p10, condensed_df_p11], ignore_index=True)
condensed_data_3_28_4angles

,A1_sd,A1_mean,A2_sd,A2_mean,A3_sd,A3_mean,A4_sd,A4_mean,x,y
0,0.455694,-66.878788,0.602957,-59.091898,1.126873,-59.759676,0.388383,-55.083102,8,3
1,1.092510,-61.473898,1.880662,-59.411015,0.365428,-55.353161,0.000000,0.000000,9,4
2,2.064072,-61.387911,0.448910,-53.873030,0.396486,-52.027826,0.450729,-55.686446,5,5
3,1.417068,-63.967837,1.179202,-58.408155,1.153554,-57.107136,1.088665,-57.945354,6,7
4,0.571581,-66.878788,1.183826,-62.419752,1.526860,-61.282731,0.869344,-56.629988,9,2
...,...,...,...,...,...,...,...,...,...,...
963,1.202640,-61.775509,0.879585,-59.486481,0.796642,-60.117639,0.884519,-58.152685,4,0
964,1.960000,-57.986019,0.997752,-62.367231,1.878586,-60.338194,0.347137,-53.748426,9,3
965,0.944575,-56.063602,1.545101,-60.063027,0.493124,-57.672605,0.669381,-61.351245,0,2
966,0.970518,-55.724282,1.786930,-63.352682,0.863907,-60.154741,1.892440,-62.235201,2,8


In [ ]:
# Specify the file path for the CSV file
output_file_path = 'dataset/condensed_data_3_28_4angles.csv'

# Export the DataFrame to a CSV file
condensed_data_3_28_4angles.to_csv(output_file_path, index=False)

In [ ]:
condensed_data_noangle = pd.concat([condensed_df_p2, condensed_df_p4], ignore_index=True)
condensed_data_noangle

,A1_sd,A1_mean,A2_sd,A2_mean,A3_sd,A3_mean,A4_sd,A4_mean,x,y
0,0.455694,-66.878788,0.602957,-59.091898,1.126873,-59.759676,0.388383,-55.083102,8,3
1,1.092510,-61.473898,1.880662,-59.411015,0.365428,-55.353161,0.000000,0.000000,9,4
2,2.064072,-61.387911,0.448910,-53.873030,0.396486,-52.027826,0.450729,-55.686446,5,5
3,1.417068,-63.967837,1.179202,-58.408155,1.153554,-57.107136,1.088665,-57.945354,6,7
4,0.571581,-66.878788,1.183826,-62.419752,1.526860,-61.282731,0.869344,-56.629988,9,2
...,...,...,...,...,...,...,...,...,...,...
237,0.875053,-64.568562,1.562406,-61.102244,0.966814,-60.059715,1.099878,-59.261398,10,5
238,0.921540,-62.662452,0.947312,-50.468678,0.909721,-57.013745,1.074675,-59.190004,1,3
239,0.317204,-67.392857,0.578601,-58.457102,0.521060,-59.233333,0.531223,-54.875048,8,3
240,1.456722,-62.959921,1.696974,-58.268918,0.698608,-56.694444,1.167166,-57.905316,6,7


In [ ]:
condensed_data_all = pd.concat([condensed_data, condensed_df_p3], ignore_index=True)
condensed_data_all

,A1_sd,A1_mean,A2_sd,A2_mean,A3_sd,A3_mean,A4_sd,A4_mean,x,y
0,1.224650,-60.473337,0.829594,-64.959278,0.985218,-61.635577,0.938979,-64.718678,7,8
1,1.293700,-64.089406,0.967864,-57.929804,0.500912,-53.741022,0.496124,-55.773512,6,2
2,2.760060,-62.622321,2.713263,-61.967645,1.257812,-63.217529,0.488384,-57.486850,10,6
3,1.220471,-60.789345,1.119790,-59.706204,0.873253,-61.671806,1.294562,-63.006389,5,9
4,1.405208,-63.094378,1.052808,-64.719266,1.599324,-65.345833,1.133481,-64.602015,10,10
...,...,...,...,...,...,...,...,...,...,...
269,1.321831,-64.643781,0.515962,-50.706226,1.428053,-56.094157,0.957545,-60.704981,3,0
270,0.545581,-59.687391,0.661370,-54.530528,1.695413,-60.731842,1.330658,-60.606650,3,0
271,0.262295,-54.666296,0.893082,-57.163565,0.601628,-54.444769,0.390757,-56.412407,5,5
272,1.861798,-60.201977,2.578376,-62.799603,0.887034,-59.588985,0.443792,-58.500096,9,5


In [ ]:
# Specify the file path for the CSV file
output_file_path = 'dataset/condensed_data_NOAngle.csv'

# Export the DataFrame to a CSV file
condensed_data_noangle.to_csv(output_file_path, index=False)


In [ ]:
%cd /content/drive/Shareddrives/RFID_23-24/12_8_2023 11x11 30 Points

/content/drive/Shareddrives/RFID_23-24/12_8_2023 11x11 30 Points


In [ ]:
%ls

'30 Points .gsheet'   P3/


In [ ]:
# Directory containing your CSV files
directory = 'P3'

# DataFrame to store the condensed data
condensed_df_p3 = pd.DataFrame()

# List of filenames (replace with your list of filenames)
P3_list = os.listdir(directory)

# Iterate over each file in the list
for filename in P3_list:
    if filename.endswith('.csv'):
        result_df = process_csv_file(filename, directory)
        if result_df is not None:
            condensed_df_p3 = pd.concat([condensed_df_p2, result_df], ignore_index=True)

# Print the condensed DataFrame
print(condensed_df_p3)

       A1_sd    A1_mean     A2_sd    A2_mean     A3_sd    A3_mean     A4_sd  \
0   1.340222 -62.269821  1.279995 -64.040962  0.816688 -57.982499  0.949120   
1   0.792111 -58.923889  0.709001 -58.558519  0.511812 -67.465741  0.905727   
2   1.501747 -56.237315  0.599992 -54.067500  1.614970 -65.210760  1.332563   
3   1.200557 -62.397318  1.196624 -59.087069  1.342572 -65.095279  0.767967   
4   0.725333 -62.936412  0.883015 -62.317741  0.677667 -65.158518  0.602634   
5   1.046681 -63.870265  0.822930 -62.206418  1.017718 -66.076852  1.024729   
6   0.690349 -62.128565  1.180407 -60.494101  1.022979 -62.044821  1.293945   
7   0.926473 -56.595139  0.375049 -51.574901  1.114801 -62.731753  0.745628   
8   1.314428 -60.464167  0.836146 -56.494676  0.748503 -57.252037  1.040490   
9   1.658485 -64.645032  1.626258 -63.303338  0.422843 -56.123172  0.856725   
10  0.870117 -62.808133  1.812249 -59.560969  1.309373 -64.520132  1.305489   
11  1.147435 -57.498285  1.297463 -51.639732  1.0286

In [ ]:
# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/12_1_2023 11x11/dataset/condensed_data_test.csv'

# Export the DataFrame to a CSV file
condensed_df_p3.to_csv(output_file_path, index=False)

In [ ]:
# Specify the file path for the CSV file
output_file_path = '/content/drive/Shareddrives/RFID_23-24/3.1 Final Setup/dataset/condensed_data_RepeatofFall.csv'

# Export the DataFrame to a CSV file
condensed_data.to_csv(output_file_path, index=False)